In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# Wczytanie Penn World Table 11.0
pwt = pd.read_stata('Dane/pwt110.dta')

# Przechowanie kodów krajów
countrycodes = pwt['countrycode']
countrycodes = countrycodes.drop_duplicates()

year_last = pwt['year'].unique()[-1]

# Ustawienie MultiIndex
pwt.set_index(['countrycode', 'year'], inplace=True)

pwt.tail()

In [ ]:
# Dane za najnowszy rok w zbiorze danych
pwt_last = pwt.xs(year_last, level='year')

gdppc	= (pwt_last['rgdpe']/pwt_last['pop']) 					# Realne PKB na osobę
cpc		= (pwt_last['csh_c']*pwt_last['rgdpe']/pwt_last['pop']) # Konsumpcja na osobę
pop		= pwt_last['pop']										# Populacja kraju

df = pd.DataFrame()

df['gdppc'] = gdppc
df['l_gdppc'] = np.log(gdppc)
df['cpc'] = cpc
df['l_cpc'] = np.log(cpc)
df['pop'] = pop
# df

# Sortowanie po populacji
df.sort_values('pop', ascending=False, inplace=True)
df.head(10)

In [ ]:
fig, ax = plt.subplots()

sns.regplot(ax=ax, x='l_gdppc', y='l_cpc', data=df, 
			scatter_kws={'s': df['pop'], 'alpha': 0.5}, line_kws={'color': 'C3'})

for country in df.iloc[:40].index:
	ax.annotate(country, (df['l_gdppc'][country], df['l_cpc'][country]), 
				ha='center', va='center', fontsize=7)

ticks = np.array([0.5, 1, 2, 5, 10, 20, 50, 100, 200])

plt.xticks(np.log(ticks*1000), [int(tick) if tick.is_integer() else f"{tick:.1f}" for tick in ticks])
plt.yticks(np.log(ticks[1:]*1000), [int(tick) if tick.is_integer() else f"{tick:.1f}" for tick in ticks[1:]])

plt.xlim(np.log(5e2), None)
plt.ylim(np.log(5e2), np.log(100_000))

plt.title('PKB vs konsumpcja na osobę w 2023')
plt.xlabel('PKB na osobę w 2023 (tys. USD z 2021)')
plt.ylabel('Konsumpcja na osobę w 2023 (tys. USD z 2021)')

plt.show()

In [ ]:
# Dane z Banku Światowego o oczekiwanej długości życia
wb = pd.read_excel('Dane/API_SP.DYN.LE00.IN_DS2_en_excel_v2_2658.xls', 
				   sheet_name='Data', skiprows=3, index_col=1)
lf = wb[str(year_last)]

# Łączenie zbiorów danych
pwt_wb = pd.concat([lf, gdppc, np.log(gdppc), pop], axis=1)
pwt_wb.columns = ['life_exp','gdppc','l_gdppc','pop']
pwt_wb = pwt_wb.dropna()

# Sortowanie po populacji
pwt_wb.sort_values('pop', ascending=False, inplace=True)
pwt_wb.head(10)

In [ ]:
fig, ax = plt.subplots()

sns.regplot(ax=ax, x='l_gdppc', y='life_exp', data=pwt_wb, 
			scatter_kws={'s': pwt_wb['pop'], 'alpha': 0.5}, line_kws={'color': 'C3'})

for country in pwt_wb.iloc[:40].index:
	ax.annotate(country, (pwt_wb['l_gdppc'][country], pwt_wb['life_exp'][country]), 
				ha='center', va='center', fontsize=7)

ticks = np.array([0.5, 1, 2, 5, 10, 20, 50, 100, 200])

plt.xticks(np.log(ticks*1000), [int(tick) if tick.is_integer() else f"{tick:.1f}" for tick in ticks])

plt.xlim(np.log(5e2), None)
plt.ylim(50, 90)

plt.title('PKB na osobę vs oczekiwana długość życia w 2023')
plt.xlabel('PKB na osobę w 2023 (tys. USD z 2021)')
plt.ylabel('Oczekiwana długość życia w 2023')

plt.show()

In [ ]:
# Histogram poziomów PKB na osobę
log_bins = np.logspace(np.log(500), np.log(200_000), num=1+10, base=np.e)

plt.hist(df['gdppc'], bins=log_bins, histtype='bar', rwidth=0.8)

plt.xscale('log')

ticks = np.array([0.5, 1, 2, 5, 10, 20, 50, 100, 200])
plt.xticks(ticks * 1000, [int(tick) if tick.is_integer() else f"{tick:.1f}" for tick in ticks])

plt.title('Histogram poziomów PKB na osobę 2023')
plt.xlabel('PKB na osobę w 2023 (tys. USD z 2021)')
plt.ylabel('Liczba krajów')

plt.show()

In [ ]:
sns.kdeplot(df['l_gdppc'], fill=True, lw=2)

ticks = np.array([0.2, 0.5, 1, 2, 5, 10, 20, 50, 100, 200])

plt.xticks(np.log(ticks*1000), [int(tick) if tick.is_integer() else f"{tick:.1f}" for tick in ticks])

plt.xlim(np.log(2e2), np.log(300_000))

plt.title('Rozkład KDE poziomów PKB na osobę w 2023')
plt.xlabel('PKB na osobę w 2023 (tys. USD z 2021)')
plt.ylabel('Gęstość krajów')

plt.show()

In [ ]:
sl = pwt_last.dropna(subset=['rgdpe','pop'])
y = sl['rgdpe']/sl['pop']
y_rel_us = 100*y/(sl['rgdpe']['USA']/sl['pop']['USA'])
pop_tot = np.sum(sl['pop'])

plt.style.use('default')
plt.hist(y_rel_us, weights=100*sl['pop']/pop_tot,
		 bins=np.arange(0, 110, 1), cumulative=True, 
		 histtype='step', linewidth=3)

plt.xlim(0, 100)
plt.ylim(0, 100)

plt.title('Populacja świata względem PKB na osobę w 2023')
plt.xlabel('PKB na osobę w porównaniu do USA w 2023')
plt.ylabel('Skumulowana część globalnej populacji (%)')

plt.grid()

plt.show()

In [ ]:
# Rozkład PKB na osobę ważonego populacją (nie krajami)

for i, year in enumerate([1960, 1990, 2023]):
	sl = pwt.xs(year, level='year').dropna(subset=['rgdpe','pop'])
	pop_tot = np.sum(sl['pop'])

	weighted = sm.nonparametric.KDEUnivariate(np.log(sl['rgdpe']/sl['pop']))
	weighted.fit(bw=0.4, fft=False, weights=sl['pop'])
	plt.plot(np.exp(weighted.support), 
			 100*weighted.density/np.sum(weighted.density), 
			 lw=3, label='{0}'.format(year))

plt.xscale('log')

ticks = np.array([0.2, 0.5, 1, 2, 5, 10, 20, 50, 100, 200])
plt.xticks(ticks * 1000, [int(tick) if tick.is_integer() else f"{tick:.1f}" for tick in ticks])
plt.yticks(np.arange(0, 3, 0.5))

plt.xlim(200, 300_000)
plt.ylim(0, 2.5)
	
plt.legend(frameon=False)

plt.title('Rozkład PKB na osobę ważonego populacją')
plt.xlabel('PKB na osobę (tys. USD z 2021)')
plt.ylabel('Udział populacji (%)')

plt.show()

In [ ]:
# Histogram geometrycznych stóp wzrostu

x_70 = pwt.xs(1970, level='year')['rgdpe']/pwt.xs(1970, level='year')['pop']

x_last = pwt_last['rgdpe']/pwt_last['pop']

g = 100*((x_last/x_70)**(1/(year_last-1970))-1)

plt.hist(g.dropna(), np.arange(1.5, 2.5, 0.5), histtype='bar', rwidth=0.8, fc='darkgrey', label='Kraje rozwinięte')
plt.hist(g.dropna(), np.arange(-10, 2, 0.5), histtype='bar', rwidth=0.8, fc='C3', label='Kraje odstające')
plt.hist(g.dropna(), np.arange(2, 10, 0.5), histtype='bar', rwidth=0.8, fc='C2', label='Kraje doganiające')

plt.xlim(-4, 8)

plt.title('Histogram stóp wzrostu PKB na osobę')
plt.xlabel('Przeciętna (geometryczna) stopa wzrostu PKB na osobę, 1970-2023 (%)')
plt.ylabel('Liczba krajów')

plt.legend(frameon=False, loc='upper left')

plt.show()

In [ ]:
# PKB na osobę w krajach rozwiniętych

(pwt.loc['USA']['rgdpe']/pwt.loc['USA']['pop']).plot(lw=2, label='Stany Zjednoczone')
(pwt.loc['GBR']['rgdpe']/pwt.loc['GBR']['pop']).plot(lw=2, label='Wielka Brytania')
(pwt.loc['CHE']['rgdpe']/pwt.loc['CHE']['pop']).plot(lw=2, label='Szwajcaria')
(pwt.loc['FRA']['rgdpe']/pwt.loc['FRA']['pop']).plot(lw=2, label='Francja')
(pwt.loc['DEU']['rgdpe']/pwt.loc['DEU']['pop']).plot(lw=2, label='Niemcy')
(pwt.loc['JPN']['rgdpe']/pwt.loc['JPN']['pop']).plot(lw=2, label='Japonia')

plt.xlim(1950, 2025)

plt.yscale('log')
plt.ylim(2500, 100000)

ticks = np.array([5, 10, 20, 50, 100])
plt.yticks(ticks*1000, ticks)

plt.legend(frameon=False, ncol=2)

plt.title('PKB na osobę w krajach rozwiniętych')
plt.xlabel('')
plt.ylabel('PKB na osobę (tys. USD z 2021)')

plt.show()

In [ ]:
# Średnie na poziomie krajów
s = np.zeros(len(countrycodes))
n = np.zeros(len(countrycodes))
y = np.zeros(len(countrycodes))
h = np.zeros(len(countrycodes))
N = np.zeros(len(countrycodes))

for i, country in enumerate(countrycodes):
	s[i] = 100*np.mean(pwt.loc[country]['csh_i'])
	n[i] = 100*np.mean(pwt.loc[country]['pop'].pct_change())
	y[i] = pwt_last.loc[country]['rgdpo']/pwt_last.loc[country]['emp']
	h[i] = pwt_last.loc[country]['hc']
	N[i] = pwt_last.loc[country]['pop']

d = {'y': y, 's': s, 'n': n, 'h': h, 'N': N}
dta = pd.DataFrame(data=d, index=countrycodes)
dta['ln_y'] = np.log(dta['y'])

# Sortowanie po populacji
dta.sort_values('N', ascending=False, inplace=True)
dta.head(10)

In [ ]:
fig, ax = plt.subplots()

sns.regplot(ax=ax, x='s', y='ln_y', data=dta.dropna(), 
			scatter_kws={'s': dta.dropna()['N'], 'alpha': 0.5}, line_kws={'color': 'C3'})

for country in dta.dropna().iloc[:40].index:
	ax.annotate(country, (dta['s'][country], dta['ln_y'][country]), 
				ha='center', va='center', fontsize=7)

ticks = np.array([0.5, 1, 2, 5, 10, 20, 50, 100, 200, 300])

plt.yticks(np.log(ticks*1000), [int(tick) if tick.is_integer() else f"{tick:.1f}" for tick in ticks])
plt.ylim(np.log(2000), np.log(300_000))

plt.xlim(5, 45)

plt.title(r'Stopa inwestycji $I/Y$ vs PKB na pracownika')
plt.xlabel('Przeciętna stopa inwestycji, 1950-2023 (%)')
plt.ylabel('PKB na pracownika w 2023 (tys. USD z 2021)')

plt.show()

In [ ]:
fig, ax = plt.subplots()

sns.regplot(ax=ax, x='n', y='ln_y', data=dta.dropna(), 
			scatter_kws={'s': dta.dropna()['N'], 'alpha': 0.5}, line_kws={'color': 'C3'})

for country in dta.dropna().iloc[:40].index:
	ax.annotate(country, (dta['n'][country], dta['ln_y'][country]), 
				ha='center', va='center', fontsize=7)

ticks = np.array([0.5, 1, 2, 5, 10, 20, 50, 100, 200, 300])

plt.yticks(np.log(ticks*1000), [int(tick) if tick.is_integer() else f"{tick:.1f}" for tick in ticks])
plt.ylim(np.log(2000), np.log(300_000))

plt.xticks(range(-2, 9))
plt.xlim(-2, 8)

plt.title(r'Tempo zmiany populacji $n$ vs PKB na pracownika')
plt.xlabel('Przeciętne tempo zmiany populacji, 1950-2023 (%)')
plt.ylabel('PKB na pracownika w 2023 (tys. USD z 2021)')

plt.show()

In [ ]:
fig, ax = plt.subplots()

sns.regplot(ax=ax, x='h', y='ln_y', data=dta.dropna(), 
			scatter_kws={'s': dta.dropna()['N'], 'alpha': 0.5}, line_kws={'color': 'C3'})

for country in dta.dropna().iloc[:40].index:
	ax.annotate(country, (dta['h'][country], dta['ln_y'][country]), 
				ha='center', va='center', fontsize=7)

ticks = np.array([0.5, 1, 2, 5, 10, 20, 50, 100, 200, 300])

plt.yticks(np.log(ticks*1000), [int(tick) if tick.is_integer() else f"{tick:.1f}" for tick in ticks])
plt.ylim(np.log(2000), np.log(300_000))

plt.xticks(range(1, 5))
plt.xlim(1, 4.2)

plt.title('Kapitał ludzki vs PKB na pracownika')
plt.xlabel('Poziom kapitału ludzkiego w 2023 (funkcja lat edukacji)')
plt.ylabel('PKB na pracownika w 2023 (tys. USD z 2021)')

plt.show()

In [ ]:
# Replikacja Mankiw, Romer and Weil (1992)
mrw = pd.read_stata('Dane/MRW1992.dta')
mrw_countries = mrw['country']
mrw.head()

In [ ]:
# Obróbka danych do regresji
mrw['y_85'] = np.log(mrw['Y85'])
mrw['s'] = np.log(mrw['invest']/100)
mrw['δ_n_g'] = np.log(0.05+mrw['pop_growth']/100)
mrw['restricted'] = mrw['s'] - mrw['δ_n_g']

In [ ]:
# Regresja na nieeksporterach ropy
mrw_N = mrw[mrw['N']==1]
print('\t Non-oil countries')
mrw_results = smf.ols('y_85 ~ s + δ_n_g', data=mrw_N).fit()
print(mrw_results.summary())

In [ ]:
# Regresja z restrykcją
print('\t Restricted regression')
print('\t Non-oil countries')
mrw_results_restricted = smf.ols('y_85 ~ restricted', data=mrw_N).fit()
print(mrw_results_restricted.summary())
print('')
print('Implied α =', mrw_results_restricted.params.iloc[1]/(1+mrw_results_restricted.params.iloc[1]))
print('')
print('Test of restriction p-value =', mrw_results.compare_f_test(mrw_results_restricted)[1])

In [ ]:
# Kapitał ludzki (stopa inwestycji w h)
mrw['s_h'] = np.log(mrw['school'])

# Regresja
print('\t Non-oil countries')
mrw_h_results = smf.ols('y_85 ~ s + δ_n_g + s_h', data=mrw[mrw['N']==1]).fit()
print(mrw_h_results.summary())

In [ ]:
# Restrykcja
mrw['restricted_h'] = mrw['s_h'] - mrw['δ_n_g']
mrw_N = mrw[mrw['N']==1]

print('\t Restricted regression')
print('\t Non-oil countries')
mrw_h_results_restricted = smf.ols('y_85 ~ restricted + restricted_h', data=mrw_N).fit()
print(mrw_h_results_restricted.summary())

α_β = ((mrw_h_results_restricted.params.iloc[1]+mrw_h_results_restricted.params.iloc[2])/
       (1+mrw_h_results_restricted.params.iloc[1]+mrw_h_results_restricted.params.iloc[2]))

print('')
print('Implied α =', mrw_h_results_restricted.params.iloc[1] * (1-α_β))
print('Implied β =', mrw_h_results_restricted.params.iloc[2] * (1-α_β))
print('')
print('Test of restriction p-value =', mrw_h_results.compare_f_test(mrw_h_results_restricted)[1])

In [ ]:
# Początkowy poziom (log) PKB na pracownika
mrw['y_60'] = np.log(mrw['Y60'])

mrw['y_85_60'] = mrw['y_85'] - mrw['y_60']

mrw_N = mrw[mrw['N']==1]

# Regresje na podpróbkach
print('\t Non-oil countries')
mrw_results_N = smf.ols('y_85_60 ~ y_60 + s + δ_n_g + s_h', data=mrw_N).fit()
print(mrw_results_N.summary())
print('')
print('Implied λ =', np.log(1+mrw_results_N.params.iloc[1])/(-25))

In [ ]:
print('\t Intermediate countries')
mrw_results_I = smf.ols('y_85_60 ~ y_60 + s + δ_n_g + s_h', data=mrw[mrw['I']==1]).fit()
print(mrw_results_I.summary())
print('')
print('Implied λ =', np.log(1+mrw_results_I.params.iloc[1])/(-25))

In [ ]:
print('\t OECD countries')
mrw_results_O = smf.ols('y_85_60 ~ y_60 + s + δ_n_g + s_h', data=mrw[mrw['O']==1]).fit()
print(mrw_results_O.summary())
print('')
print('Implied λ =', np.log(1+mrw_results_O.params.iloc[1])/(-25))

In [ ]:
# Wykresy konwergencji

par = mrw_results_N.params

plt.scatter(mrw_N['y_60'], 100*mrw_N['y_85_60']/25)

plt.title('Konwergencja bezwarunkowa')
plt.xlabel('Logarytm PKB na osobę dorosłą w 1960')
plt.ylabel('Stopa wzrostu PKB na osobę w latach 1960-85')

plt.show()

###
plt.scatter(mrw_N['y_60'], 
            100/25*(mrw_N['y_85_60']
                    -par.iloc[2]*(mrw_N['s']-mrw_N['s'].mean())
                    -par.iloc[3]*(mrw_N['δ_n_g']-mrw_N['δ_n_g'].mean())))

plt.title('Konwergencja warunkowa: s i n')
plt.xlabel('Logarytm PKB na osobę dorosłą w 1960')
plt.ylabel('Stopa wzrostu PKB na osobę w latach 1960-85')

plt.show()

###
plt.scatter(mrw_N['y_60'], 
            100/25*(mrw_N['y_85_60']
                    -par.iloc[2]*(mrw_N['s']-mrw_N['s'].mean())
                    -par.iloc[3]*(mrw_N['δ_n_g']-mrw_N['δ_n_g'].mean())
                    -par.iloc[4]*(mrw_N['s_h']-mrw_N['s_h'].mean())))

plt.title('Konwergencja warunkowa: s, n i s_h')
plt.xlabel('Logarytm PKB na osobę dorosłą w 1960')
plt.ylabel('Stopa wzrostu PKB na osobę w latach 1960-85')

plt.show()

### Replikacja na PWT dla lat 1985+

In [ ]:
# Replikacja na PWT dla lat 1985+ (włączając eksporterów ropy)

s = np.zeros(len(countrycodes))
n = np.zeros(len(countrycodes))
h = np.zeros(len(countrycodes))
y_85 = np.zeros(len(countrycodes))
y_23 = np.zeros(len(countrycodes))
N = np.zeros(len(countrycodes))

for i, country in enumerate(countrycodes):
    s[i] = np.log(np.mean(pwt.loc[country]['csh_i'][35:]))
    n[i] = np.log(0.05+np.mean(pwt.loc[country]['emp'][35:].pct_change()))
    h[i] = np.mean(pwt.loc[country]['hc'][35:])
    y_85[i] = np.log(pwt.loc[country, 1985]['rgdpo']/pwt.loc[country, 1985]['emp'])
    y_23[i] = np.log(pwt.loc[country, 2023]['rgdpo']/pwt.loc[country, 2023]['emp'])
    N[i] = pwt.loc[country, 2023]['pop']

d = {'y_23': y_23, 'y_85': y_85, 's': s, 'δ_n_g': n, 's_h': np.log(np.log(h)*10), 'N': N}
mrw_rep = pd.DataFrame(data=d, index=countrycodes)
mrw_rep['y_23_85'] = mrw_rep['y_23'] - mrw_rep['y_85']
# mrw_rep.head()

# Sortowanie po populacji
mrw_rep.sort_values('N', ascending=False, inplace=True)
mrw_rep.head(10)

In [ ]:
# Eksporterzy ropy i małe kraje do wykluczenia
mrw[mrw['N'] != 1]['country'].unique()

In [ ]:
oil_exporters = ['GAB', 'GMB', 'GIN', 'GNB', 'LSO', 'SWZ', 
                 'AFG', 'BHR', 'IRN', 'IRQ', 'KWT', 'OMN', 
                 'SAU', 'TWN', 'ARE', 'YEM', 'CYP',
                 'ISL', 'LUX', 'MLT', 'BRB', 'GUY', 'SUR', 
                 'FJI',
                 # Błąd w danych o csh_i?
                 'NGA']

mrw_rep_dta = mrw_rep[~mrw_rep.index.isin(oil_exporters)]
mrw_rep_dta['restricted'] = mrw_rep_dta['s'] - mrw_rep_dta['δ_n_g']
mrw_rep_dta['restricted_h'] = mrw_rep_dta['s_h'] - mrw_rep_dta['δ_n_g']
mrw_rep_dta

In [ ]:
mrw_rep_results = smf.ols('y_23 ~ s + δ_n_g + s_h', data=mrw_rep_dta).fit()
print(mrw_rep_results.summary())

In [ ]:
# Restrykcja
print('\t Restricted regression')
mrw_rep_results_restr = smf.ols('y_23 ~ restricted + restricted_h', data=mrw_rep_dta).fit()
print(mrw_rep_results_restr.summary())

α_β = ((mrw_rep_results_restr.params.iloc[1]+mrw_rep_results_restr.params.iloc[2])/
       (1+mrw_rep_results_restr.params.iloc[1]+mrw_rep_results_restr.params.iloc[2]))

print('')
print('Implied α =', mrw_rep_results_restr.params.iloc[1] * (1-α_β))
print('Implied β =', mrw_rep_results_restr.params.iloc[2] * (1-α_β))
print('')
# Tu restrykcja szkodzi, widać zresztą po spadku R2
print('Test of restriction p-value =', mrw_rep_results.compare_f_test(mrw_rep_results_restr)[1])

In [ ]:
mrw_rep_dta['conv'] = 100/(2023-1985)*(mrw_rep_dta['y_23_85']
                    -par.iloc[2]*(mrw_rep_dta['s']-mrw_rep_dta['s'].mean())
                    -par.iloc[3]*(mrw_rep_dta['δ_n_g']-mrw_rep_dta['δ_n_g'].mean())
                    -par.iloc[4]*(mrw_rep_dta['s_h']-mrw_rep_dta['s_h'].mean()))

plt.scatter(mrw_rep_dta['y_85'], mrw_rep_dta['conv'])

plt.title('Konwergencja warunkowa: s, n i s_h')
plt.xlabel('Logarytm PKB na pracownika w 1985')
plt.ylabel('Stopa wzrostu PKB na pracownika w latach 1985-2023')

plt.ylim(-2, 7)

plt.show()

In [ ]:
fig, ax = plt.subplots()

sns.regplot(ax=ax, x='y_85', y='conv', data=mrw_rep_dta.dropna(), 
			scatter_kws={'s': mrw_rep_dta.dropna()['N'], 'alpha': 0.5}, line_kws={'color': 'C3'})

for country in mrw_rep_dta.dropna().iloc[:40].index:
	ax.annotate(country, (mrw_rep_dta['y_85'][country], mrw_rep_dta['conv'][country]), 
				ha='center', va='center', fontsize=7)

ticks = np.array([1, 2, 5, 10, 20, 50, 100, 200])

plt.xticks(np.log(ticks*1000), [int(tick) if tick.is_integer() else f"{tick:.1f}" for tick in ticks])
plt.xlim(np.log(1000), np.log(150_000))

plt.title('Konwergencja warunkowa w latach 1985-2023: s, n i s_h')
plt.xlabel('PKB na pracownika w 1985')
plt.ylabel('Stopa wzrostu PKB na pracownika w latach 1985-2023')

plt.ylim(-2, 7)

plt.show()

In [ ]:
mrw_results_conv = smf.ols('y_23_85 ~ y_85 + s + δ_n_g + s_h', data=mrw_rep_dta).fit()
print(mrw_results_conv.summary())
print('')
print('Implied λ =', np.log(1+mrw_results_conv.params.iloc[1])/(1985-2023))